In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Adding system path
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent.parent))
# sys.path

In [ ]:
# Set to show warnings only once
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Setting up displays
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import math
from plotly import tools
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate
color_scheme=["red","blue","green","orange","purple","brown","pink","gray","olive","cyan","darkviolet","goldenrod","darkgreen","chocolate","lawngreen"]

In [ ]:
# Setting up autoreload for libs
%load_ext autoreload
%autoreload 2
%aimport optiml.backend.warehouse_profile

In [ ]:
# Initialize connection to Snowflake and set analysis date
from optiml.connection import SnowflakeConnConfig
connection = SnowflakeConnConfig(accountname='jg84276.us-central1.gcp',warehousename="XSMALL_WH").create_connection()
# Initialize local environment
import os
cache_dir = os.path.expanduser('~/data/kiva')
# Initialize query library
from optiml.backend.warehouse_profile import WarehouseProfile
wqlib =WarehouseProfile(connection, 'KIV', cache_dir)
sdate = '2022-09-12'
edate = '2022-10-12'
print(f"The analysis is carried our for date range {sdate} to {edate}")

### Warehouse load analysis

In [ ]:
df=wqlib.wh_queued_load_ts(start_date="2022-09-12",end_date="2022-10-12",wh_name="PROD_WH",delta='hour')
df.head()


In [ ]:

# fig.append_trace(go.bar(df, x=df["start_time_truncated"], y=[df["avg_queued_load"], df["avg_running"]], title=" DEV_WH", color_discrete_sequence=color_scheme),row=1,col=1) 

fig=px.bar(df, x=df["start_time_truncated"], y=[df["avg_queued_load"], df["avg_running"]], title=" DEV_WH", color_discrete_sequence=color_scheme)
# Range the width has to be updated in
fig.show()

In [ ]:

df1=wqlib.wh_queued_load_ts(start_date="2022-09-12",end_date="2022-10-12",wh_name="PROD_WH",delta='hour')
df2=wqlib.wh_credits(sdate,edate)
trace1 = go.Bar(
    x=df1["start_time_truncated"], y=df1["avg_queued_load"],
    name='Queued Load',marker=dict(color='rgb(222,0,0)')
)
trace2 = go.Bar(
    x=df1["start_time_truncated"], y=df1["avg_running"],
    name='Running load'
)
trace3 = go.Scatter(
  x=df2['start_time_truncated']
  ,y=df2['credits_used']
  ,fill='tozeroy'
  ,name='Credits consumed'
  )


fig=make_subplots(rows=1,cols=1,shared_yaxes=True)


fig.append_trace(trace3, 1,1)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2,1,1)


fig['layout'].update(height=600, width=2000)
fig.show()